In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_targets_scored.csv


In [2]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import zero_one_loss
from sklearn.metrics import log_loss

In [3]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

In [4]:
#replace "-" by "_" for attributes
train_fea_processed = train_features.replace("-", "_")
print(train_fea_processed.shape)
train_fea_processed.head()

(23814, 876)


,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [5]:
# encode cp_type and cp_dose to binary
train_fea_processed['cp_type'] = train_fea_processed['cp_type'].replace({'trt_cp':1, 'ctl_vehicle':0})
train_fea_processed['cp_dose'] = train_fea_processed['cp_dose'].replace({'D1':1, 'D2':0})
train_fea_processed

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,1,24,1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,1,72,1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,1,48,1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,1,48,1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,1,72,0,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,1,24,0,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,1,24,0,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,0,48,0,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,1,24,1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [6]:
# encode cp_time to binary
train_fea_processed = pd.get_dummies(train_fea_processed, columns=["cp_time"])
train_fea_processed

,sig_id,cp_type,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-93,c-94,c-95,c-96,c-97,c-98,c-99,cp_time_24,cp_time_48,cp_time_72
0,id_000644bb2,1,1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,1,0,0
1,id_000779bfc,1,1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371,0,0,1
2,id_000a6266a,1,1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,0,1,0
3,id_0015fd391,1,1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,0,1,0
4,id_001626bd3,1,0,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,1,0,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,...,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680,1,0,0
23810,id_fffb70c0c,1,0,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,...,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689,1,0,0
23811,id_fffc1c3f4,0,0,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,-0.2541,...,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808,0,1,0
23812,id_fffcb9e7c,1,1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,...,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740,1,0,0


In [7]:
# swap cp_time columns.
train_fea_processed=pd.concat([train_fea_processed.iloc[:,0:2],train_fea_processed.iloc[:,875:878], train_fea_processed.iloc[:,2:875]], axis=1)
train_fea_processed

,sig_id,cp_type,cp_time_24,cp_time_48,cp_time_72,cp_dose,g-0,g-1,g-2,g-3,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,1,1,0,0,1,1.0620,0.5577,-0.2479,-0.6208,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,1,0,0,1,1,0.0743,0.4087,0.2991,0.0604,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,1,0,1,0,1,0.6280,0.5817,1.5540,-0.0764,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,1,0,1,0,1,-0.5138,-0.2491,-0.2656,0.5288,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,1,0,0,1,0,-0.3254,-0.4009,0.9700,0.6919,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,1,1,0,0,0,0.1394,-0.0636,-0.1112,-0.5080,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,1,1,0,0,0,-1.3260,0.3478,-0.3743,0.9905,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,0,0,1,0,0,0.3942,0.3756,0.3109,-0.7389,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,1,1,0,0,1,0.6660,0.2324,0.4392,0.2044,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [8]:
#replace "-" by "_" for label dataset
train_targets_scored_processed = train_targets_scored.replace("-", "_")
print(train_targets_scored_processed.shape)
train_targets_scored_processed.head()

(23814, 207)


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#drop the samples having all zero in label profiles
train_targets_scored_processed["RowTotal"] = train_targets_scored_processed.sum(axis=1)
train_targets_scored_processed

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,RowTotal
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23810,id_fffb70c0c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23811,id_fffc1c3f4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
train_targets_scored_processed_short = train_targets_scored_processed[train_targets_scored_processed['RowTotal'] > 0]
train_targets_scored_processed_short

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,RowTotal
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,id_001762a82,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,id_0020d0484,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23807,id_fff7e6992,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23808,id_fff8c2444,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23809,id_fffb1ceed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23810,id_fffb70c0c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
data = train_fea_processed.merge(train_targets_scored_processed_short, on='sig_id')
data

,sig_id,cp_type,cp_time_24,cp_time_48,cp_time_72,cp_dose,g-0,g-1,g-2,g-3,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,RowTotal
0,id_000644bb2,1,1,0,0,1,1.0620,0.5577,-0.2479,-0.6208,...,0,0,0,0,0,0,0,0,0,1
1,id_000a6266a,1,0,1,0,1,0.6280,0.5817,1.5540,-0.0764,...,0,0,0,0,0,0,0,0,0,3
2,id_001626bd3,1,0,0,1,0,-0.3254,-0.4009,0.9700,0.6919,...,0,0,0,0,0,0,0,0,0,1
3,id_001762a82,1,1,0,0,1,-0.6111,0.2941,-0.9901,0.2277,...,0,0,0,0,0,0,0,0,0,1
4,id_0020d0484,1,0,1,0,1,0.2711,0.5133,-0.1327,2.5950,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14442,id_fff7e6992,1,0,1,0,1,0.1420,-0.3696,-0.0093,-0.2495,...,0,0,0,0,0,0,0,0,0,1
14443,id_fff8c2444,1,0,0,1,1,0.1608,-1.0500,0.2551,-0.2239,...,0,0,0,0,0,0,0,0,0,1
14444,id_fffb1ceed,1,1,0,0,0,0.1394,-0.0636,-0.1112,-0.5080,...,0,0,0,0,0,0,0,0,0,1
14445,id_fffb70c0c,1,1,0,0,0,-1.3260,0.3478,-0.3743,0.9905,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Input Layer : The input features are specified via the “input_dim” argument in the first hidden layer and denoted by "n_inputs".
# Hidden Layer : hidden layer has 878 nodes..  "ReLU" activation function is used for hidden layer.
# Output Layer : Output layer has 207 nodes equivalent to total number of labels. "Sigmoid" activation function is used for each node in the output layer,
# The fit of model will be measured by binary cross-entropy loss and the Adam version of stochastic gradient descent.

In [24]:
# Get the Data
X = data.iloc[:,1:878]
X = np.array(X)
y = data.iloc[:,878:1084]
y = np.array(y)
    
# evaluate a model using repeated k-fold cross-validation

Accuracy_results = list()
Hamming_Loss_results = list()
Log_Loss_results = list()
Zero_One_Loss_results = list()
yhat_clas = []

# define evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# enumerate folds
for train_ix, test_ix in cv.split(X):
    # prepare data
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    
    # define model
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    model = Sequential()
    model.add(Dense(1400, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(1400, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(1400, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(1400, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(1400, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(1400, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(1400, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(1000, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(700, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(400, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    
    # fit (train) model
    #model.fit(X_train, y_train, verbose=0, epochs=35,)
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=0, batch_size=35)
     
    # evaluate the model
    _, train_acc = model.evaluate(X_train, y_train, verbose=0)
    _, test_acc = model.evaluate(X_test, y_test, verbose=0)
    
    # predict (probabilities)
    yhat_prob = model.predict(X_test)
    yhat_prob = yhat_prob.astype(np.float64)

    # convert probabilities to class
    yhat_clas = yhat_prob.copy()
    # class 1
    yhat_clas[yhat_clas >= 0.5] = 1
    # class 0
    yhat_clas[yhat_clas < 0.5] = 0
    
    # convert to integer type
    yhat_clas = yhat_clas.astype(int)
    
    #Accuracy
    acc = accuracy_score(y_test, yhat_clas)
    
    # Hamming Loss
    hamloss = hamming_loss(y_test, yhat_clas)
    
    # Log Loss
    logloss = log_loss(y_test, yhat_prob)
    
    # Zero_One Loss
    zer_one = zero_one_loss(y_test, yhat_clas)
    
    print ('=====================================')
        
   
    print('Train_acc: %.3f, Test_acc: %.3f' % (train_acc, test_acc))
    
    print("Hamming_Loss:",'>%.3f' % hamloss)
    Hamming_Loss_results.append(hamloss)
    
    print("Log_Loss:",'>%.3f' % logloss)
    Log_Loss_results.append(logloss)
    
    print("Zero_One_Loss:",'>%.3f' % zer_one)
    Zero_One_Loss_results.append(zer_one)

Train: 0.224, Test: 0.210
Accuracy : >0.166
Hamming_Loss: >0.005
Log_Loss: >4.332
Zero_One_Loss: >0.834


KeyboardInterrupt: 

In [ ]:
# summarize performance
print('Zero_One: %.3f (%.3f)' % (mean(Zero_One_results), std(Zero_One_results)))
print('Hamming_Loss: %.3f (%.3f)' % (mean(Ham_loss_results), std(Ham_loss_results)))
print('Log_Loss: %.3f (%.3f)' % (mean(Log_loss_results), std(Log_loss_results)))